In [1]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from datasets import load_dataset
# from evaluate import load
import json

# output_model_dir = 'Vi-Mong-mBART'
# model_checkpoint = "AI_models/model_hmong_vietnamese_mt_mbart"

/home/vietnq2/Projects/Personal/LearnMong/learn-vietnamese-mong/.env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pwd

/home/vietnq2/Projects/Personal/LearnMong/quizz/scripts


In [3]:
# Load model and tokenizer
model = MBartForConditionalGeneration.from_pretrained("../AI_models/model_hmong_vietnamese_mt_mbart")
tokenizer = MBart50TokenizerFast.from_pretrained("../AI_models/tokenizer_hmong_vietnamese_mt_mbart")

tokenizer_json = json.loads(tokenizer._tokenizer.to_str())


In [7]:
tokenizer.add_special_tokens({'additional_special_tokens': ['hmn_VN']})

1

In [13]:
tokenizer.lang_code_to_id['hmn_VN']

KeyError: 'hmn_VN'

In [8]:
tokenizer.get_added_vocab()['hmn_VN']

250246

In [9]:
tokenizer.src_lang = 'hmn_VN'

In [12]:
tokenizer.decode(tokenizer('hello')['input_ids'])

'hmn_VN hello</s>'

## Prepare data

In [64]:
from datasets import load_dataset
import os


class DataLoader:
    """
    A simple DataLoader for text data stored in csv files.
    """

    def __init__(self, data_path, columns=None, batch_size=32):
        """
        Initialize the DataLoader with the path to the training data and batch size.

        Args:
            data_path (str): Path to the folder containing csv files.
            columns (list): List of columns to load from the csv files.
        """
        self.batch_size = batch_size
        self.columns = columns
        self._load_data(data_path)

    def _load_data(self, data_path: str):
        """
        Load multiple csv files from a given path and create a Hugging Face Dataset.
        """

        try:
            assert os.path.exists(data_path)
        except AssertionError:
            print(f"Data path does not exist: {data_path}")
            raise FileNotFoundError(f"Data path does not exist: {data_path}")

        data_files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith(".csv")]
        self.dataset = load_dataset(
            "csv",
            data_files=data_files,
            split="train",
        )
        if self.columns:
            self.dataset = self.dataset.select_columns(self.columns)
        print(f"Loaded {len(self.dataset)} samples from {self.dataset}.")

    def __iter__(self):
        """
        Iterate over the dataset and yield batches of text data.
        """
        for batch in self.dataset.iter(batch_size=self.batch_size):
            if self.columns and len(self.columns) == 1:
                yield batch[self.columns[0]]
            else:
                yield batch
                # yield {col: batch[col] for col in self.columns}


In [75]:
dataloader = DataLoader(data_path='../data', batch_size=2)

Loaded 778 samples from Dataset({
    features: ['mong', 'vietnamese'],
    num_rows: 778
}).


In [76]:
for x in dataloader:
    print(x)
    break

{'mong': ['kêl', 'saz'], 'vietnamese': ['thấp', 'cao']}


## Train LLM

In [77]:
tokenizer.src_lang = "hm"
tokenizer.tgt_lang = "vi"

In [78]:
max_input_length = 128
max_target_length = 128

def preprocess_function(examples, from_lang="mong", to_lang="vietnamese"):
    print(examples)
    print(examples[from_lang])
    print(examples[to_lang])
    model_inputs = tokenizer(
        text=examples[from_lang],
        text_target=examples[to_lang],
        max_length=max_input_length, truncation=True)
    print(model_inputs)
    return model_inputs

In [79]:
dataloader.dataset = dataloader.dataset.map(lambda x: preprocess_function(x), batched=True, remove_columns=dataloader.dataset.column_names)
dataloader.dataset

Map:   0%|          | 0/778 [00:00<?, ? examples/s]

{'mong': ['kêl', 'saz', 'lux', 'yaz', 'jông', 'đuz', 'cơưv', 'môngl', 'naox', 'uô', 'nôngs', 'shôngz', 'ntông', 'trăngz hlâu', 'trăngz cơư', 'trăngz tras', 'luz tsêr', 'luz cơưv', 'luz trôngx', 'lênhx', 'lênhx nav', 'lênhx txir', 'lênhx txir zơưv', 'đeiv qaox', 'đeiv ntơưr', 'đeiv blôngx', 'txux plâuz hâu', 'txux xur', 'chaor', 'puôk', 'cxaz', 'yiv', 'đangx', 'lôngx têl', 'phưz', 'langv phưx', 'đar', 'khaoz', 'jas', 'pluôv', 'hnuz', 'hmao', 'hlik (hli)', 'shông', 'ntiv', 'cheix', 'tav', 'tik', 'sênhv', 'txôngl', 'jaol', 'xar', 'xinhr', 'zangv', 'txeir', 'tuz', 'nxeik', 'nhuôs', 'tix', 'cưr', 'vêr', 'ndâus', 'nhangz', 'vâur', 'zơưv', 'pus', 'tsêr', 'têz', 'lax', 'đêr', 'keiz', 'ntơưr', 'đêx', 'shuv xinhz', 'cưk ntông', 'tsêr khu maoz', 'đha', 'nhaoz', 'zang zus', 'uô si', 'nưl đha', 'tâu naox', 'tâu hâuk', 'tâu cơưv', 'tâu uô si', 'tsuv tuô', 'tsuv tuv', 'trâus maoz', 'trâus  phuôv', 'tsuv ntâuk', 'az', 'kăngz', 'saz', 'greiv', 'yaz', 'tsêr nor jông hưngr', 'tsêr nor jông jông', 'tsêr n

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 778
})

In [15]:
tokenizer.decode(
    tokenized_dataset['train']['input_ids'][2],
    use_source_tokenizer=True
), tokenizer.decode(
    tokenized_dataset['train']['labels'][2],
    use_source_tokenizer=False
)

('hmn_VN lub zog</s>', 'vi_VN sức mạnh</s>')

In [16]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [17]:
import numpy as np

metric = load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [18]:
from transformers import Seq2SeqTrainingArguments

source_lang = "hmn_VN"
target_lang = "vi_VN"

batch_size = 32
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_strategy="epoch",
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=30,
    predict_with_generate=True,
    fp16=True,
    # push_to_hub=True,
)

/home/vietnq2/anaconda3/envs/LearnMong/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_49957/3606200012.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [20]:
# turn of warning
from transformers import logging
logging.set_verbosity_error()

trainer.train()

  0%|          | 0/390 [00:00<?, ?it/s]

{'loss': 5.6562, 'grad_norm': 22.556230545043945, 'learning_rate': 1.953846153846154e-05, 'epoch': 1.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 2.6418263912200928, 'eval_bleu': 2.9175, 'eval_gen_len': 5.102, 'eval_runtime': 1.1532, 'eval_samples_per_second': 84.984, 'eval_steps_per_second': 3.469, 'epoch': 1.0}
{'loss': 2.3898, 'grad_norm': 13.926077842712402, 'learning_rate': 1.8871794871794873e-05, 'epoch': 2.0}


  0%|          | 0/4 [00:00<?, ?it/s]

...

/home/vietnq2/anaconda3/envs/LearnMong/lib/python3.12/site-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


{'loss': 0.0456, 'grad_norm': 2.8344736099243164, 'learning_rate': 2.0512820512820514e-07, 'epoch': 30.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 3.1520814895629883, 'eval_bleu': 2.9279, 'eval_gen_len': 5.7653, 'eval_runtime': 1.2522, 'eval_samples_per_second': 78.265, 'eval_steps_per_second': 3.194, 'epoch': 30.0}
{'train_runtime': 184.4892, 'train_samples_per_second': 63.418, 'train_steps_per_second': 2.114, 'train_loss': 0.6294279514214931, 'epoch': 30.0}


TrainOutput(global_step=390, training_loss=0.6294279514214931, metrics={'train_runtime': 184.4892, 'train_samples_per_second': 63.418, 'train_steps_per_second': 2.114, 'total_flos': 358431412125696.0, 'train_loss': 0.6294279514214931, 'epoch': 30.0})

In [25]:
dataset
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 390
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 98
    })
})

In [34]:
tokenized_dataset['test'][0]['input_ids'], tokenizer.decode(tokenized_dataset['test'][0]['input_ids'])

([250175, 74185, 250109, 876, 298, 170, 4981, 2],
 'hmn_VN tsi ntsi b koj dua</s>')

In [49]:
import torch

idx = 45
text = dataset['test']['from_lang_text'][idx]
print('Expect:', dataset['test']['to_lang_text'][idx])

input_tokenized = tokenizer(text, add_special_tokens=True)
print(f"{input_tokenized['input_ids']}  --  {tokenizer.decode(input_tokenized['input_ids'])}")
input_ids = torch.tensor(input_tokenized['input_ids']).unsqueeze(0).to('cuda')
attention_mask = torch.tensor(input_tokenized['attention_mask']).unsqueeze(0).to('cuda')

# Generate outputs
output_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=256
)

outputs = [tokenizer.decode(ids, skip_special_tokens=True) for ids in output_ids]
outputs

Expect: cam kết hàng ngày
[250175, 311, 334, 552, 177, 6, 5782, 250117, 75, 11, 250067, 6, 5829, 2]  --  hmn_VN kev cog lus txh ua hn ub</s>


['cam kết dài hạn']